In [2]:
def getInput():
    return open("../input/day_5.txt").read()

def getInputLines():
    return getInput().split("\n")

# Day 5 Part 1

parsing: /`(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)`/gms

In [3]:
import re

text = getInput()

# text = """
# seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4
# """.strip()

# match all numbers, first match is seed line, subsequent matches are the maps
seeds, *maps = [
    [int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)]
    for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)
    ]
# sort ranges in maps based on their start number
maps = [sorted(r, key=lambda x: x[1]) for r in
    [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]]

# main
locs = []
for seed in seeds:
    temp = seed
    for map in maps:
        for (dst, src, span) in map:
            if src <= temp < src + span:
                temp = dst + (temp - src)
                break
    locs.append(temp)
min(locs)
# 331445006


331445006

# Day 5 Part 2

In [40]:
import re

text = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

# text = getInput()

seeds, *maps = [[int(v) for v in re.findall(r"\d+", match.groups()[1], flags=re.MULTILINE)] for match in re.finditer(r"(^\n?[a-zA-Z\-\:\s]+)((\d+[.\s]?)+)", text, flags=re.MULTILINE)]
seeds = [seeds[i:i+2] for i in range(0, len(seeds), 2)]
maps = [sorted(r, key=lambda x: x[1]) for r in [[m[i:i+3] for i in range(0, len(m), 3)] for m in maps]]

# I'm gonna have to write real code for this one...

def extract_range(seed_range, map_range):
    ranges = []
    
    seed_start, seed_length = seed_range
    seed_end = seed_start + seed_length - 1
    map_dst, map_start, map_length = map_range
    map_end = map_start + map_length - 1

    # no intersection
    if (seed_start > map_end) or (map_start > seed_end):
        ranges.append([seed_start, seed_length])
    else:
        split_start = max(map_start, seed_start)
        split_end = min(map_end, seed_end)
        split_length = split_end - split_start
        ds = split_start - seed_start
        de = split_end - seed_end
        
        # map range starts within the seed range
        if ds > 0:
            ranges.append([seed_start, ds + 1])
        # map range ends within the seed range
        if de < 0:
            ranges.append([split_end + 1, -de])

        seed_dst = split_start + (map_dst - map_start)

        ranges.append([seed_dst, split_length + 1])
    return ranges



seed_ranges = [[s,l] for s,l in seeds]
for m in maps:
    # print("New map, input:", seed_ranges)
    for mr in m:
        ranges = []
        for sr in seed_ranges:
            olds, oldl = sr
            # print("Testing seed range:", sr)
            new_ranges = extract_range(sr, mr)
            # print("Result:", new_ranges)
            ranges.extend(new_ranges)
        seed_ranges = ranges
        
print(min(seed_ranges, key=lambda x:x[0])[0])

        
# 24092691 too high
# 20522066 too high
# 19624706

46


# Debugging: 1
running `extract_range((6, 20), (0, 10, 5))`
i get `[[6, 5], [15, 11], [0, 5]]`

while it should be
- 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25
- 10, 11, 12, 13, 14 -> 0, 1, 2, 3, 4
- 6, 7, 8, 9 + 0, 1, 2, 3, 4 + 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25
- [6, 4] + [0, 5] + [15, 11]

In [23]:
extract_range([57, 13], [49, 53, 8])

[[61, 9], [53, 4]]

# Debugging: 2

running `extract_range([57, 13], [49, 53, 8])` gives `[[61, 8], [53, 4]]` which doesn't make sense

- sr = 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69
- mr = 53, 54, 55, 56, 57, 58, 59, 60
- r0 = 53, 54, 55, 56
- r1 = 61, 62, 63, 64, 65, 66, 67, 68, (missing: 69)

In [36]:
maps1

[[[52, 50, 48], [50, 98, 2]],
 [[39, 0, 15], [0, 15, 37], [37, 52, 2]],
 [[42, 0, 7], [57, 7, 4], [0, 11, 42], [49, 53, 8]],
 [[88, 18, 7], [18, 25, 70]],
 [[81, 45, 19], [68, 64, 13], [45, 77, 23]],
 [[1, 0, 69], [0, 69, 1]],
 [[60, 56, 37], [56, 93, 4]]]